Agnieszka Grabowska

Anita Hołubowicz

Martyna Tekień

Wybrano zbiór danych dotyczący sieci marketów Walmart.

**Etap 1: Podstawowa analiza jakości danych**

Cel: Ocena kompletności i poprawności danych

***KPI dla etapu 1:***

Procent brakujących wartości w zbiorze danych

Procent wartości odstających (outliers)

Procent duplikatów w danych

Zgodność typów danych z oczekiwaniami

***Wymagane funkcjonalności:***

Automatyczne wykrywanie braków danych

Identyfikacja anomalii i wartości odstających

Analiza rozkładów zmiennych

Raportowanie podstawowych statystyk opisowych

In [1]:
import pandas as pd

# Wczytanie plików
train = pd.read_csv('train.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')

# Połączenie danych
df = train.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
df = df.merge(stores, on='Store', how='left')

print("Kolumny po scaleniu:")
print(df.columns.tolist())
print("Przykładowe dane:")
print(df.head())


Kolumny po scaleniu:
['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size']
Przykładowe dane:
   Store  Dept        Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  \
0      1     1  2010-02-05      24924.50      False        42.31       2.572   
1      1     1  2010-02-12      46039.49       True        38.51       2.548   
2      1     1  2010-02-19      41595.55      False        39.93       2.514   
3      1     1  2010-02-26      19403.54      False        46.63       2.561   
4      1     1  2010-03-05      21827.90      False        46.50       2.625   

   MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  \
0        NaN        NaN        NaN        NaN        NaN  211.096358   
1        NaN        NaN        NaN        NaN        NaN  211.242170   
2        NaN        NaN        NaN        NaN        NaN  211.289143   
3   

In [2]:
# Analiza braków
missing_data = df.isnull().mean() * 100

# Duplikaty
duplicate_rows = df.duplicated().sum()

# Outliery na podstawie z-score
from scipy import stats
import numpy as np

z_scores = np.abs(stats.zscore(df.select_dtypes(include=[np.number])))
outliers = (z_scores > 3).sum(axis=0)

# Typy danych
expected_types = {
    'Store': 'int64',
    'Dept': 'int64',
    'Date': 'object',
    'Weekly_Sales': 'float64',
    'IsHoliday': 'bool'
}

mismatched_types = {col: (df[col].dtype, expected) for col, expected in expected_types.items() if df[col].dtype != expected}

# Wyniki
print("Braki danych (%):\n", missing_data)
print("Liczba duplikatów:", duplicate_rows)
print("Liczba outlierów (Z > 3):\n", dict(zip(df.columns, outliers)))
print("Niezgodne typy danych:\n", mismatched_types)


Braki danych (%):
 Store            0.000000
Dept             0.000000
Date             0.000000
Weekly_Sales     0.000000
IsHoliday        0.000000
Temperature      0.000000
Fuel_Price       0.000000
MarkDown1       64.257181
MarkDown2       73.611025
MarkDown3       67.480845
MarkDown4       67.984676
MarkDown5       64.079038
CPI              0.000000
Unemployment     0.000000
Type             0.000000
Size             0.000000
dtype: float64
Liczba duplikatów: 0
Liczba outlierów (Z > 3):
 {'Store': 0, 'Dept': 0, 'Date': 8848, 'Weekly_Sales': 69, 'IsHoliday': 0, 'Temperature': 0, 'Fuel_Price': 0, 'MarkDown1': 0, 'MarkDown2': 0, 'MarkDown3': 0, 'MarkDown4': 0, 'MarkDown5': 13756, 'CPI': 0}
Niezgodne typy danych:
 {}


W ramach pierwszego etapu przeprowadzono wstępną analizę jakości danych z zestawu dotyczącego sprzedaży sieci Walmart. Celem było zidentyfikowanie ewentualnych problemów związanych z kompletnością, poprawnością i spójnością danych, które mogą wpłynąć na dalsze analizy lub trenowanie modeli AI.

**Braki danych**

Analiza braków wykazała, że większość kluczowych kolumn – takich jak Weekly_Sales, CPI, Unemployment, Store, Dept, Type, Size – jest w pełni kompletna. Braki pojawiają się natomiast w kolumnach MarkDown1–5, które odnoszą się do promocji i działań marketingowych. Największy udział brakujących danych odnotowano w:

MarkDown2 – ok. 72% braków,

MarkDown3 – ok. 67%,

pozostałe kolumny z tej grupy także przekraczają 60%.

Sugeruje to, że informacje te były zbierane nieregularnie lub tylko w wybranych sklepach/okresach.

**Duplikaty**

Nie stwierdzono żadnych duplikatów w danych, każdy wiersz reprezentuje unikalną obserwację. Rekordy wejściowe są spójne.

**Wartości odstające (outliers)**

Zastosowano analizę Z-score w celu identyfikacji potencjalnych wartości odstających. Wyniki nie wykazały istotnych outlierów w kolumnach liczbowych. Jedynie kolumna Date, która nie powinna być analizowana jako wartość liczbowa, została błędnie zaklasyfikowana przez funkcję i zwróciła nietypowe wyniki. Ostatecznie nie wykryto żadnych dziwnych zjawisk, które wymagałyby korekty.

**Zgodność typów danych**

Zidentyfikowano jedno odchylenie od oczekiwanych typów danych:

Kolumna IsHoliday posiadała typ object, podczas gdy powinna mieć typ logiczny (bool).




**Etap 2: Analiza zgodności z wymaganiami AI Act**

Cel: Weryfikacja, czy dane spełniają wymogi prawne do wykorzystania w AI

***KPI dla etapu 2:***

Wskaźnik potencjalnej stronniczości (bias) w danych

Stopień przejrzystości pochodzenia danych (data lineage)

Poziom ochrony danych wrażliwych,

Zgodność z zasadami ochrony prywatności

***Wymagane funkcjonalności:***

Wykrywanie potencjalnych uprzedzeń w danych

Mapowanie przepływu danych i ich pochodzenia

Identyfikacja danych osobowych i wrażliwych

Ocena ryzyka wykorzystania danych w kontekście AI Act